In [1]:
from __future__ import print_function, division
import pprint
import os 
import fnmatch
#import errno
import os.path as osp
import json
#import re
import glob as gb
from six import string_types

import utils
reload(utils)

<module 'utils' from 'utils.pyc'>

In [2]:
pp = pprint.PrettyPrinter(indent=2)

In [3]:
#- testing

test_case_105a = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_sub*_model.json"
}
test_case_105b = {
    'base_dir': '/home/jb/data/bids/ds105',
    'model_pattern': "ds-105_level-all_model.json"
}
test_case_005 = {
    'base_dir': '/home/jb/data/bids/ds005',
    'model_pattern': "*_model.json"
}

test_case = test_case_005
#test_case = test_case_105a
#test_case = test_case_105b

base_dir, model_pattern = test_case['base_dir'], test_case['model_pattern'] 

all_jsons = utils.glob_recursive(base_dir, model_pattern)
print(all_jsons)
json_model=all_jsons[0]

['/home/jb/data/bids/ds005/models/ds-005_model.json']


In [4]:
data = utils.get_json_model_Ydata(all_jsons[0], level='Run', verbose=utils.VERB['info'])

base dir /home/jb/data/bids/ds005
json_model /home/jb/data/bids/ds005/models/ds-005_model.json


In [5]:
pp.pprint(data[:min(10, len(data))])

[ '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-03/func/sub-03_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-02_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-01_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-08/func/sub-08_task-mixedgamblestask_run-03_bold.nii.gz',
  '/home/jb/data/bids/ds005/sub-12/func/sub-12_task-mixedgamblestask_run-02_bold.nii.gz']


In [6]:
model_pattern = "*_model.json"
model_data = utils.get_funcs_models(base_dir, model_pattern, verbose=utils.VERB['none'])

In [7]:
datafile = data[0]
model_dict = model_data[datafile]
pp.pprint(model_dict)

{ u'Columns': { u'dummy': { u'HRFmodelling': u'Gamma',
                            u'Level': -1,
                            u'Variable': u'respcat',
                            u'prm_modulation': u'RT'},
                u'gain': { u'HRFmodelling': u'Gamma',
                           u'Level': 0,
                           u'Variable': u'respcat',
                           u'prm_modulation': u'RT'},
                u'loss': { u'HRFmodelling': u'Gamma',
                           u'Level': 1,
                           u'Variable': u'respcat',
                           u'prm_modulation': u'RT'}},
  u'Contrasts': { u'pos_vs_neg_contrast': { u'Columns': [u'gain', u'loss'],
                                            u'Weights': [1, -1]}},
  u'DependentVariable': u'task-mixedgamblestask',
  u'Level': u'Run'}


In [8]:
import pandas as pd
tsv_file = datafile.split('_bold.nii')[0] + '_events.tsv'
assert osp.isfile(tsv_file), "{} is not a file ".format(tsv_file)
print(tsv_file)

/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_events.tsv


In [9]:
dic = utils._get_json_dict_from_tsv_file(tsv_file)
#utils._get_tsv_lines(dic, 'trial_type', 'chair')

In [10]:
model_dict = model_data[datafile]
print(model_dict.keys())
regressors = model_dict['Columns']
pp.pprint(regressors)

[u'Contrasts', u'DependentVariable', u'Columns', u'Level']
{ u'dummy': { u'HRFmodelling': u'Gamma',
              u'Level': -1,
              u'Variable': u'respcat',
              u'prm_modulation': u'RT'},
  u'gain': { u'HRFmodelling': u'Gamma',
             u'Level': 0,
             u'Variable': u'respcat',
             u'prm_modulation': u'RT'},
  u'loss': { u'HRFmodelling': u'Gamma',
             u'Level': 1,
             u'Variable': u'respcat',
             u'prm_modulation': u'RT'}}


In [11]:
condition_names, list_dicts = utils.get_run_conditions(datafile, model_dict)

In [12]:
condition_names

[u'dummy', u'gain', u'loss']

In [13]:
#pp.pprint(list_dicts)
nipype_info = utils.get_nipype_run_info(datafile, model_dict, verbose=utils.VERB['none'])

In [14]:
print(datafile)
#pp.pprint(nipype_info)
print(nipype_info)

/home/jb/data/bids/ds005/sub-01/func/sub-01_task-mixedgamblestask_run-01_bold.nii.gz
{'prm_modulation': [[0.0], [1.6369999999999998, 1.8220000000000001, 1.206, 1.321, 1.2150000000000001, 1.0469999999999999, 1.137, 1.0820000000000001, 1.585, 1.806, 1.6000000000000001, 1.5980000000000001, 1.3680000000000001, 2.302, 1.6869999999999998, 1.383, 1.7849999999999999], [1.7929999999999999, 1.3159999999999998, 1.6699999999999999, 1.232, 1.5019999999999998, 1.925, 1.0609999999999999, 1.079, 1.4770000000000001, 1.599, 1.405, 1.272, 0.90000000000000002, 1.3180000000000001, 1.367, 1.5659999999999998, 1.3600000000000001, 1.4330000000000001, 0.82400000000000007, 0.90300000000000002, 1.169, 1.0960000000000001, 2.2000000000000002, 0.77599999999999991, 1.3360000000000001, 1.3620000000000001, 1.786, 1.107, 1.2430000000000001, 1.1120000000000001, 1.2350000000000001, 0.86599999999999999, 1.6269999999999998, 1.0, 1.526, 1.2509999999999999, 2.2799999999999998, 1.242, 1.0109999999999999, 1.0429999999999999, 1.